In [1]:
from pynq import Overlay, allocate ## Used to interface with Xilinx Zynq hardware, allowing work in PL with Python.
import cv2 ##  Imports the OpenCV library, which is commonly used for image processing tasks.
from PIL import Image, ImageFilter ## Used for opening, manipulating, and saving many different image file formats.
import numpy as np ## Provides support for arrays and matrices, along with a collection of mathematical functions.
import io, os ## Used for handling various types of I/O and interacting with the operating system.
ol = Overlay('Sobel_V.bit') ## Implements a Sobel filter in the FPGA using the bit file.

In [2]:
dma = ol.axi_dma_0 ## Name definition for the DMA in the system.
tr_val = ol.axi_gpio_0.channel1[0:8] ## Name definition for the threshold value by using the AXI GPIO in the system.

In [3]:
image = cv2.imread("Carry.jpg") ## Read an image file and stores it in the variable image.
tr_val.write(90) ## Write threshold value.
gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY) ## converts the original image to a grayscale image. The result is stored in gray_image.
new_width = 1280 ## Write width  value.
new_height = 720 ## Write height value.
resized_image = cv2.resize(gray_image, (new_width, new_height)) ##  resizes the gray_image to the specified dimensions (1280x720). The resized image is stored in resized_image.
cv2.imwrite('Carry_new_size.jpg', resized_image) ## saves the resized image.
image_size = (new_height, new_width) ## saves the new_height and new_width in one variable.
in_buffer = allocate(shape=(image_size), dtype=np.uint8, cacheable=True) ## allocate memory buffers for input,set to the shape defined by image_size and are of type uint8 (8-bit unsigned integers).
out_buffer = allocate(shape=(image_size), dtype=np.uint8, cacheable=True) ## allocate memory buffers for output,set to the shape defined by image_size and are of type uint8 (8-bit unsigned integers).
output_image= np.copy(out_buffer) ## copy size.
in_buffer[:] = resized_image[:] ## Copies the contents of the resized_image into the in_buffer.

In [4]:
dma.recvchannel.stop()  # Stop the DMA if it’s running
dma.recvchannel.start()  # Reset the DMA instance
dma.sendchannel.stop()  # Stop the DMA if it’s running
dma.sendchannel.start()  # Reset the DMA instance
dma.recvchannel.transfer(out_buffer) ## Start sending using the DMA the image to Sobel filter IP and wait to the end of the process. 
dma.sendchannel.transfer(in_buffer) ## Start sending using the DMA the image from DDR to Sobel filter IP and wait to the end of the process.
dma.recvchannel.wait()
dma.sendchannel.wait()

output_image[:] = out_buffer[:]
cv2.imwrite('Carry_edges(th=90).jpg', output_image) ## Saving the processed image.

del in_buffer ## Delete the buffers. 
del out_buffer